In [12]:
import inspect

# Specify the LLM Endpoint
# Now, let's prompt Raven!
API_URL = "http://nexusraven.nexusflow.ai"
headers = {"Content-Type": "application/json"}


def raven_post(payload):
    """
    Sends a payload to a TGI endpoint.
    """
    import requests

    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()


def query_raven(prompt):
    """
    This function sends a request to the TGI endpoint to get Raven's function call.
    This will not generate Raven's justification and reasoning for the call, to save on latency.
    """
    import requests

    output = raven_post(
        {
            "inputs": prompt,
            "parameters": {
                "temperature": 0.001,
                "stop": ["<bot_end>"],
                "do_sample": False,
                "max_new_tokens": 2048,
                "return_full_text": False,
            },
        }
    )
    call = output[0]["generated_text"].replace("Call:", "").strip()
    return call


def clean_docstring(docstring):
    if docstring is not None:
        # Remove leading and trailing whitespace
        docstring = docstring.strip()
    return docstring


def construct_prompt(raven_msg, functions):
    full_prompt = ""
    for function in functions:
        signature = inspect.signature(function)
        docstring = function.__doc__
        prompt = f'''Function:\n{function.__name__}{signature}\n"""{clean_docstring(docstring)}"""'''
        full_prompt += prompt + "\n\n"
    full_prompt += f"""User Query: {raven_msg}<human_end>"""
    return full_prompt

In [2]:
import requests

url = "https://v2.jokeapi.dev/joke/Any?safe-mode&type=twopart"

response = requests.get(url)

print(response.json()["setup"])
print(response.json()["delivery"])

What are bits?
Tiny things left when you drop your computer down the stairs.


In [3]:
response.json()

{'error': False,
 'category': 'Programming',
 'type': 'twopart',
 'setup': 'What are bits?',
 'delivery': 'Tiny things left when you drop your computer down the stairs.',
 'flags': {'nsfw': False,
  'religious': False,
  'political': False,
  'racist': False,
  'sexist': False,
  'explicit': False},
 'id': 211,
 'safe': True,
 'lang': 'en'}

In [5]:
import requests


def give_joke(category: str):
    """
    Joke categories. Supports: Any, Misc, Programming, Pun, Spooky, Christmas.
    """

    url = f"https://v2.jokeapi.dev/joke/{category}?safe-mode&type=twopart"
    response = requests.get(url)
    print(response.json()["setup"])
    print(response.json()["delivery"])

In [6]:
USER_QUERY = "Hey! Can you get me a joke for this december?"

In [13]:
raven_functions = f'''
def give_joke(category : str):
    """
    Joke categories. Supports: Any, Misc, Programming, Dark, Pun, Spooky, Christmas.
    """

User Query: {USER_QUERY}<human_end>
'''
call = query_raven(raven_functions)

In [14]:
exec(call)

What does Santa suffer from if he gets stuck in a chimney?
Claustrophobia!


### Writing a tool that uses OpenAPI APIs

In [15]:
!wget https://raw.githubusercontent.com/open-meteo/open-meteo/main/openapi.yml

--2024-06-20 16:41:05--  https://raw.githubusercontent.com/open-meteo/open-meteo/main/openapi.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8000::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13300 (13K) [text/plain]
Saving to: ‘openapi.yml’

openapi.yml         100%[===================>]  12.99K  --.-KB/s    in 0.005s  

2024-06-20 16:41:05 (2.39 MB/s) - ‘openapi.yml’ saved [13300/13300]



In [16]:
import yaml
import json

# Read the content of the file
with open("openapi.yml", "r") as file:
    file_content = file.read()
file_content = file_content.replace("int\n", "number\n")
file_content = file_content.replace("float\n", "number\n")
data = yaml.safe_load(file_content)

data["servers"] = [{"url": "https://api.open-meteo.com"}]

with open("openapi.json", "w") as file:
    json_content = json.dump(data, file)

$ pipx install openapi-python-client --include-deps

In [20]:
!pipx run openapi-python-generator openapi.json ./api_specification_main/

⚠️ Found a space in the home path. We heavily discourage this, due to multiple
    incompatibilities. Please check our docs for more information on this, as
    well as some pointers on how to migrate to a different home path.
⡿ installing openapi-python-generatorGenerating data from openapi.json


In [21]:
from api_specification_main.services.WeatherForecastAPIs_service\
    import get_v1forecast

In [22]:
user_query = "Hey how is the current weather and windspeed in New York?"

In [24]:
import inspect
signature = inspect.signature(get_v1forecast)
signature

<Signature (latitude: float, longitude: float, hourly: Optional[List[str]] = None, daily: Optional[List[str]] = None, current_weather: Optional[bool] = None, temperature_unit: Optional[str] = None, wind_speed_unit: Optional[str] = None, timeformat: Optional[str] = None, timezone: Optional[str] = None, past_days: Optional[int] = None, api_config_override: Optional[api_specification_main.api_config.APIConfig] = None) -> Dict[str, Any]>

In [25]:
docstring = """
Requires the latitude and longitude.
Set current_weather to True to get the weather.
Set hourly or daily based on preference.
"""

raven_prompt = f'''
Function:
{get_v1forecast.__name__}{signature}
"""{docstring}"""

User Query: {user_query}<human_end>'''

print(raven_prompt)


Function:
get_v1forecast(latitude: float, longitude: float, hourly: Optional[List[str]] = None, daily: Optional[List[str]] = None, current_weather: Optional[bool] = None, temperature_unit: Optional[str] = None, wind_speed_unit: Optional[str] = None, timeformat: Optional[str] = None, timezone: Optional[str] = None, past_days: Optional[int] = None, api_config_override: Optional[api_specification_main.api_config.APIConfig] = None) -> Dict[str, Any]
"""
Requires the latitude and longitude.
Set current_weather to True to get the weather.
Set hourly or daily based on preference.
"""

User Query: Hey how is the current weather and windspeed in New York?<human_end>


In [27]:
call = query_raven(raven_prompt)
print (call)

get_v1forecast(latitude=40.7128, longitude=-74.0060, current_weather=True)


In [28]:
eval(call)

{'latitude': 40.710335,
 'longitude': -73.99307,
 'generationtime_ms': 0.10001659393310547,
 'utc_offset_seconds': 0,
 'timezone': 'GMT',
 'timezone_abbreviation': 'GMT',
 'elevation': 32.0,
 'current_weather_units': {'time': 'iso8601',
  'interval': 'seconds',
  'temperature': '°C',
  'windspeed': 'km/h',
  'winddirection': '°',
  'is_day': '',
  'weathercode': 'wmo code'},
 'current_weather': {'time': '2024-06-21T00:30',
  'interval': 900,
  'temperature': 27.4,
  'windspeed': 8.9,
  'winddirection': 153,
  'is_day': 1,
  'weathercode': 0}}